In [1]:
import mido
import numpy as np
import binascii
import os, sys, hashlib


In [68]:
def sysex_message(patch_number, channel):
    import dx7
    # get the 155 bytes for the patch number from the C extension
    patch_data = dx7.unpack(patch_number)
    # generate the twos complement checksum for the patch data 
    # from these dudes fighting w/ each other about who has the best programming skills sigh 
    # https://yamahamusicians.com/forum/viewtopic.php?t=6864
    check = ~sum(patch_data) + 1 & 0x7F

    # Generate the sysex message
    byte_count = 155 # always 155 bytes of patch information (the operator-on message is only for live mode)
    msb = byte_count / 127
    lsb = (byte_count % 127) - 1
    return [0x43, channel, 0, msb, lsb] + patch_data + [check]

def sysex_message_from_bytearray(patch_data, channel=1): # what is channel?

    # generate the twos complement checksum for the patch data 
    # from these dudes fighting w/ each other about who has the best programming skills sigh 
    # https://yamahamusicians.com/forum/viewtopic.php?t=6864
    check = ~sum(patch_data) + 1 & 0x7F

    # Generate the sysex message
    byte_count = 155 # always 155 bytes of patch information (the operator-on message is only for live mode)
    msb = byte_count / 127
    lsb = (byte_count % 127) - 1
    return [0x43, channel, 0, msb, lsb] + patch_data + [check]




def unpack_packed_patch(p):
    # Input is a 128 byte thing from compact.bin
    # Output is a 156 byte thing that the synth knows about
    o = [0]*156
    for op in range(6):
        o[op*21:op*21 + 11] = p[op*17:op*17+11]
        leftrightcurves = p[op*17+11]
        o[op * 21 + 11] = leftrightcurves & 3
        o[op * 21 + 12] = (leftrightcurves >> 2) & 3
        detune_rs = p[op * 17 + 12]
        o[op * 21 + 13] = detune_rs & 7
        o[op * 21 + 20] = detune_rs >> 3
        kvs_ams = p[op * 17 + 13]
        o[op * 21 + 14] = kvs_ams & 3
        o[op * 21 + 15] = kvs_ams >> 2
        o[op * 21 + 16] = p[op * 17 + 14]
        fcoarse_mode = p[op * 17 + 15]
        o[op * 21 + 17] = fcoarse_mode & 1
        o[op * 21 + 18] = fcoarse_mode >> 1
        o[op * 21 + 19] = p[op * 17 + 16]
    
    o[126:126+9] = p[102:102+9]
    oks_fb = p[111]
    o[135] = oks_fb & 7
    o[136] = oks_fb >> 3
    o[137:137+4] = p[112:112+4]
    lpms_lfw_lks = p[116]
    o[141] = lpms_lfw_lks & 1
    o[142] = (lpms_lfw_lks >> 1) & 7
    o[143] = lpms_lfw_lks >> 4
    o[144:144+11] = p[117:117+11]
    o[155] = 0x3f
    
    
    # Clamp the unpacked patches to a known max. 
    maxes =  [
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc6
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc5
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc4
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc3
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc2
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc1
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, # pitch eg rate & level 
        31, 7, 1, 99, 99, 99, 99, 1, 5, 7, 48, # algorithm etc
        126, 126, 126, 126, 126, 126, 126, 126, 126, 126, # name
        127 # operator on/off
    ]
    for i in range(156):
        if(o[i] > maxes[i]): o[i] = maxes[i]
        if(o[i] < 0): o[i] = 0
    return o

def pack_unpacked_patch(o): # (reverse function)
    # Input is a 128 byte thing from compact.bin
    # Output is a 156 byte thing that the synth knows about
    p = [0]*128
    
    for op in range(6):
        
        #o[op*21:op*21 + 11] = p[op*17:op*17+11]
        p[op*17:op*17+11] = o[op*21:op*21 + 11] # apparently right (quick scan)
        
        """
        leftrightcurves = p[op*17+11]
        o[op * 21 + 11] = leftrightcurves & 3
        o[op * 21 + 12] = (leftrightcurves >> 2) & 3
        """
        p[op*17+11] = o[op * 21 + 11] # not sure?
        
        """
        detune_rs = p[op * 17 + 12]
        o[op * 21 + 13] = detune_rs & 7
        o[op * 21 + 20] = detune_rs >> 3
        """
        p[op * 17 + 12] = o[op * 21 + 20] << 3
        
        """
        kvs_ams = p[op * 17 + 13]
        o[op * 21 + 14] = kvs_ams & 3
        o[op * 21 + 15] = kvs_ams >> 2
        """
        p[op * 17 + 13] = o[op * 21 + 15] << 2
        
        #o[op * 21 + 16] = p[op * 17 + 14]
        p[op * 17 + 14] = o[op * 21 + 16]
        
        """
        fcoarse_mode = p[op * 17 + 15]
        o[op * 21 + 17] = fcoarse_mode & 1
        o[op * 21 + 18] = fcoarse_mode >> 1
        """
        p[op * 17 + 15] = o[op * 21 + 18] << 1
        
        #o[op * 21 + 19] = p[op * 17 + 16]
        p[op * 17 + 16] = o[op * 21 + 19]

    #o[126:126+9] = p[102:102+9]
    p[102:102+9] = o[126:126+9] 
    
    """
    oks_fb = p[111]
    o[135] = oks_fb & 7
    o[136] = oks_fb >> 3
    """
    p[111] = o[136] << 3
    
    #o[137:137+4] = p[112:112+4]
    p[112:112+4] = o[137:137+4]
    
    """
    lpms_lfw_lks = p[116]
    o[141] = lpms_lfw_lks & 1
    o[142] = (lpms_lfw_lks >> 1) & 7
    o[143] = lpms_lfw_lks >> 4
    """
    p[116] = o[143] << 4
    
    #o[144:144+11] = p[117:117+11]
    p[117:117+11] = o[144:144+11]
    
    #o[155] = 0x3f

    """# Clamp the unpacked patches to a known max. 
    maxes =  [
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc6
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc5
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc4
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc3
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc2
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, # osc1
        3, 3, 7, 3, 7, 99, 1, 31, 99, 14,
        99, 99, 99, 99, 99, 99, 99, 99, # pitch eg rate & level 
        31, 7, 1, 99, 99, 99, 99, 1, 5, 7, 48, # algorithm etc
        126, 126, 126, 126, 126, 126, 126, 126, 126, 126, # name
        127 # operator on/off
    ]
    for i in range(156):
        if(o[i] > maxes[i]): o[i] = maxes[i]
        if(o[i] < 0): o[i] = 0"""
    
    return p


def convert_unpacked_to_compact(): # (reverse function)
    # Take a compact.bin and make it unpacked.bin
    f = bytearray(open('unpacked.bin', "rb").read())
    o = open('compact_bis.bin','wb')
    num_patches = len(f)/156
    for patch in range(int(num_patches)):
        patch_data = f[patch*156:patch*156+156]
        packed = pack_unpacked_patch(patch_data)
        o.write(bytearray(packed))
    o.close()
    
def convert_compact_to_unpacked():
    # Take a compact.bin and make it unpacked.bin
    f = bytearray(open("compact.bin",'rb').read())
    o = open("unpacked.bin", "wb")
    num_patches = int(len(f)/128)
    print('num_patches',num_patches)
    for patch in range(num_patches):
        patch_data = f[patch*128:patch*128+128]
        unpacked = unpack_packed_patch(patch_data)
        o.write(bytearray(unpacked))
    o.close()


In [84]:
convert_compact_to_unpacked() # Unpack the largest existing dataset of unique DX7 voices (from https://github.com/bwhitman/learnfm)

with open('unpacked.bin', 'rb') as f:
    unpacked = np.array(bytearray(f.read())) # read the unpacked .bin file
    
presets = unpacked.reshape((-1,156))[:,:145] # each voice is encoded in 156 bytes but only the first 145 ones represent sound parameters
print("The dataset contains "+str(presets.shape[0])+" voices")
print(presets.shape)


def preprocess_dataset(data):
    maxs = data.max(axis=0)
    data = data/maxs # normalize
    return data, maxs

presets, maxs = preprocess_dataset(presets)
np.savez('presets_and_maxs', presets, maxs)


num_patches 31380
The dataset contains 31380 voices
(31380, 145)


In [81]:

# print(compact.shape)
# print(compact_bis.shape)
# convert_compact_to_unpacked()
# convert_unpacked_to_compact()

# compact = load_data("compact.bin")
# compact_bis = load_data("compact_bis.bin")

# for i in range(20):
#     save_preset(np.array(compact[i]),"reconstruct/orig_"+str(i)+".syx")
#     save_preset(np.array(compact_bis[i]),"reconstruct/bis_"+str(i)+".syx")
#     print("\n\n")
#     for j in range(128):
#         if compact[i,j] != compact_bis[i,j] :
#             print(j, compact[i,j], compact_bis[i,j])
    
#     save_preset(np.array(packed[i]),"reconstruct/"+str(i)+".syx")